In [ ]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS//')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert
from matplotlib.gridspec import GridSpec

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

In [ ]:
savefig = 0  # 1 = yes, 0 = no
figdir = '../../Figures/Retrieval/ret_sensitivity'
form = 'png'
if savefig == 1:
    cF.createFolder('%s' %figdir)

In [ ]:
################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Observations-hour_minutes'

################################################################
### Retrieval 
txt_retrieval = '../../Data/Retrieved_SWC'

################################################################

In [ ]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir, txt_filename),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')

DateHour = Haukeli['Date-Hour(UTC)']

RR_1 = Haukeli['RR_1'].astype(float)           # Precipitation (1 hour); [mm]
precip, dt_precip, hour_precip, day_precip, month_precip, year_precip = obsDat.valid_values(RR_1, DateHour)

In [ ]:
acc_dofe = []
acc_dofe.append(0.0)
for i in range(0,np.asarray(precip[21]).shape[0]):
    di = np.nansum([acc_dofe[i], precip[21][i]], axis=0)
    acc_dofe.append(di)

In [ ]:
# read in the retrieval B8 observation file, for 22 Dec
Retrieval_B8 = pd.read_csv('%s/20161222_B8_Agg2.txt' %txt_retrieval, sep=';', header = 0)

In [ ]:
acc_B8 = Retrieval_B8['acc_day'].astype(float)

In [ ]:
# read in the retrieval B6 observation file, for 22 Dec
Retrieval_B6 = pd.read_csv('%s/20161222_B6_Agg2.txt' %txt_retrieval, sep=';', header = 0)

In [ ]:
acc_B6 = Retrieval_B6['acc_day'].astype(float)

In [ ]:
year, month,ini_day = 2016,12,22

In [ ]:
no2 = np.array([131,156,45])/255.

In [ ]:
fig = plt.figure(figsize=(12,7))
ax = plt.axes()
# Vertical line to show end of day
ax.axvline(0,color = spagh.vert_col, linewidth = 3)
ax.axvline(24,color = spagh.vert_col, linewidth = 3)
ax.axvline(48,color = spagh.vert_col, linewidth = 3)

### DOUBLE FENCE ####
ax.plot(np.arange(0,len(acc_dofe)), acc_dofe, marker = 'H', markersize = 20,
         linestyle = 'None', label = 'double fence')
### B6 ACCUMULATION ####
ax.plot(np.arange(0,len(acc_B6)/60,1/60), acc_B6, color = no2,
         linestyle = (0, (3, 1, 1, 1)), label = 'retrieval_B6', linewidth=6)

### B6 ACCUMULATION ####
ax.plot(np.arange(0,len(acc_B8)/60,1/60), acc_B8,
         linestyle = (0, (3, 1, 1, 1)), label = 'retrieval_B8', linewidth=6)

#############
### fine tuning
lgd = ax.legend(loc='upper left', #bbox_to_anchor=(0.5, -0.37),
          fancybox=True, shadow=True, #ncol=3, 
                fontsize=spagh.label_fs)
frame = lgd.get_frame()
frame.set_facecolor('white')

# yaxis
ax.set_ylabel('precipitation amount [mm]', fontsize=spagh.label_fs)
ax.set_ylim(-0.5,80)
T = np.arange(0,90,10)
ax.set_yticks(T)
ax.set_yticklabels(T,fontsize = spagh.tick_fs)

# xaxis
#a = lead_time_sfc[0][0:48]
ax.set_xlim(-0.5,len(acc_dofe))
ax.set_xlabel('time', fontsize=spagh.label_fs)
ax.set_xticks(np.arange(0,len(acc_dofe),3))
    
xdays = ['%s-%s-%s' %(year,month,ini_day), '',
        6,'',12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+1), '',
        6,'', 12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+2)]
ax.set_xticklabels(xdays, rotation = 25, fontsize = spagh.tick_fs)

plt.tight_layout()

fig_name = '%s%s%s.png' %(year,month,ini_day)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

In [ ]:
per_diff_B6 = (acc_B6[12*60]-acc_dofe[12])/acc_dofe[12]*100
per_diff_B8 = (acc_B8[12*60]-acc_dofe[12])/acc_dofe[12]*100

In [ ]:
print('double fence acc: 12h',acc_dofe[12])
print('retrieval B6 acc: 12h',acc_B6[12*60])
print('retrieval B8 acc: 12h',acc_B8[12*60])

In [ ]:
print('percent Diff B6: 12h',np.asarray(per_diff_B6))
print('percent Diff B8: 12h',np.asarray(per_diff_B8))

In [ ]:
per_diff_B6 = (acc_B6[24*60-1]-acc_dofe[24])/acc_dofe[24]*100
per_diff_B8 = (acc_B8[24*60-1]-acc_dofe[24])/acc_dofe[24]*100

In [ ]:
print('double fence acc: 24h',acc_dofe[23])
print('retrieval B6 acc: 24h',acc_B6[24*60-1])
print('retrieval B8 acc: 24h',acc_B8[24*60-1])

In [ ]:
print('percent Diff B6: 24h',np.asarray(per_diff_B6))
print('percent Diff B8: 24h',np.asarray(per_diff_B8))